In [ ]:
#git clone https://github.com/adiravishankara/Dreambooth-Stable-Diffusion.git

In [ ]:

#!conda env create -f environment.yaml


In [ ]:
#!conda activate ldm
#if you get the distutils has not attribute version error
#!pip install setuptools==59.5.0

In [ ]:
### SKIP
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.0
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
#!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install wget

In [ ]:
## FOR THE ORIGINAL VERSION SKIP
!pip install wget
uname = 'adididwhawt'
password = '2r8iTP$a2j-jrPE'
!wget --http-user={uname} --http-password={password} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

## FOR THE FULL EMA VERSION
# !wget --http-user={} --http-password={} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt

In [ ]:
!mv sd-v1-4.ckpt model.ckpt

In [ ]:
### SKIP
dataset="woman_ddim"
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p outputs/txt2img-samples/samples/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* outputs/txt2img-samples/samples/{dataset}

In [ ]:
!apt install -y unzip
!unzip regularization.zip
!uzip training_samples.zip

In [ ]:
# START THE TRAINING
project_name = "PM-Aiyana"

# MAX STEPS
# It's how long you want your training to go.
# If you're seeing this message, I'm literally at my computer right now fixing this up:
max_training_steps = 1500

class_word = "woman"  # << match this word to the class word from regularization images above
reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization/woman_ddim"

try:
    !rm -rf training_samples/.ipynb_checkpoints
except:
    print("No checkpoints for ipynb")

!python "main.py" --base configs/stable-diffusion/v1-finetune_unfrozen.yaml -t --actual_resume "model.ckpt" --reg_data_root {reg_data_root} -n {project_name} --gpus 0,1 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_samples" --max_training_steps {max_training_steps} --class_word class_word --no-test

In [ ]:
directory_paths = !ls -d logs/*


In [ ]:
# This version should automatically prune around 10GB from the ckpt file
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
!python "prune_ckpt.py" --ckpt {last_checkpoint_file}

In [ ]:
last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
training_samples = !ls training_samples
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_samples)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + class_word + "_class_word.ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{file_name}